 # Evaluate Baselines (Airfoil use case)

The goal of this notebook is to demonstrate how we can evaluate the results of a baseline on a given benchmark for a fully custom model with its own training procedure. 

In this case, LIPS is used mainly for the evaluation with a few exceptions (scaler, rebuilding the dataset from the prediction...)

We will show how to run the baseline and evaluate it on the Benchmark of the competition. The methodology is akin to the one described in the AirfRANS paper for a MLP model.

### Prerequisites

Install the LIPS framework if it is not already done. For more information look at the LIPS framework [Github repository](https://github.com/IRT-SystemX/LIPS) 

In [ ]:
# !pip install -r requirements.txt
# or 
# !pip install -U .

Install the AirfRANS package

In [ ]:
# !pip install airfrans

#### Import required LIPS packages

In [ ]:
from lips import get_root_path
from lips.benchmark.airfransBenchmark import AirfRANSBenchmark
from lips.dataset.airfransDataSet import download_data
from lips.dataset.scaler.standard_scaler_iterative import StandardScalerIterative

#### Import other packages

In [ ]:
import os
import time
import random
import math 

from tqdm import tqdm
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch.nn import BatchNorm1d, Identity
from torch.nn import Linear
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data

Define the configuration files path, that aim to describe specific caracteristics of the use case or the augmented simulator.

In [ ]:
LIPS_PATH = get_root_path()
DIRECTORY_NAME = 'Dataset'
BENCHMARK_NAME = "DEFAULT"
LOG_PATH = LIPS_PATH + "lips_logs.log"
BENCH_CONFIG_PATH = os.path.join("airfoilConfigurations","benchmarks","confAirfoil.ini") 

directory_name='Dataset'

First, we define a simple MLP class

In [ ]:
class MLP(torch.nn.Module):
    def __init__(self, channel_list, dropout = 0.,
                 batch_norm = True, relu_first = False):
        super().__init__()
        assert len(channel_list) >= 2
        self.channel_list = channel_list
        self.dropout = dropout
        self.relu_first = relu_first

        self.lins = torch.nn.ModuleList()
        for dims in zip(self.channel_list[:-1], self.channel_list[1:]):
            self.lins.append(Linear(*dims))

        self.norms = torch.nn.ModuleList()
        for dim in zip(self.channel_list[1:-1]):
            self.norms.append(BatchNorm1d(dim, track_running_stats = False) if batch_norm else Identity())

        self.reset_parameters()

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()
        for norm in self.norms:
            if hasattr(norm, 'reset_parameters'):
                norm.reset_parameters()


    def forward(self, x: Tensor) -> Tensor:
        x = self.lins[0](x)
        for lin, norm in zip(self.lins[1:], self.norms):
            if self.relu_first:
                x = x.relu_()
            x = norm(x)
            if not self.relu_first:
                x = x.relu_()
            x = F.dropout(x, p = self.dropout, training = self.training)
            x = lin.forward(x)
        return x


    def __repr__(self) -> str:
        return f'{self.__class__.__name__}({str(self.channel_list)[1:-1]})'

and we consider the following neural network model

In [ ]:
class NN(torch.nn.Module):
    def __init__(self, hparams, encoder, decoder):
        super(NN, self).__init__()
        self.nb_hidden_layers = hparams['nb_hidden_layers']
        self.size_hidden_layers = hparams['size_hidden_layers']
        self.bn_bool = hparams['bn_bool']
        self.activation = nn.ReLU()
        self.encoder = encoder
        self.decoder = decoder
        self.dim_enc = hparams['encoder'][-1]
        self.nn = MLP([self.dim_enc] + [self.size_hidden_layers]*self.nb_hidden_layers + [self.dim_enc], batch_norm = self.bn_bool)

    def forward(self, data):
        z = self.encoder(data.x)        
        z = self.nn(z)
        z = self.decoder(z)
        return z


Now, we are in position to implement the augmented simulator. There are two points worth mentionning in this approach:

- The neural network model relies on a representation that allow it to see each CFD simulation within a dataset individually (see `process_dataset` method below for more details)
- The training involves a subsampling of a constant number of points within each simulation

In [ ]:
class AugmentedSimulator():
    def __init__(self,benchmark,**kwargs):
        self.name = "AirfRANSSubmission"
        chunk_sizes=benchmark.train_dataset.get_simulations_sizes()
        scalerParams={"chunk_sizes":chunk_sizes}
        self.scaler = StandardScalerIterative(**scalerParams)
        self.hparams = kwargs
        use_cuda = torch.cuda.is_available()
        self.device = 'cuda:0' if use_cuda else 'cpu'
        if use_cuda:
            print('Using GPU')
        else:
            print('Using CPU')

        encoder = MLP(self.hparams['encoder'], batch_norm = False)
        decoder = MLP(self.hparams['decoder'], batch_norm = False)
        self.model = NN(self.hparams, encoder, decoder)

    def train(self,train_dataset):
        train_dataset = self.process_dataset(dataset=train_dataset,training=True)
        global_train(self.device, train_dataset, self.model, self.hparams,criterion = 'MSE_weighted')

    def predict(self,dataset,**kwargs):
        print(dataset)
        test_dataset = self.process_dataset(dataset=dataset,training=False)
        self.model.eval()
        avg_loss_per_var = np.zeros(4)
        avg_loss = 0
        avg_loss_surf_var = np.zeros(4)
        avg_loss_vol_var = np.zeros(4)
        avg_loss_surf = 0
        avg_loss_vol = 0
        iterNum = 0

        predictions=[]
        with torch.no_grad():
            for data in test_dataset:        
                data_clone = data.clone()
                data_clone = data_clone.to(self.device)
                out = self.model(data_clone)

                targets = data_clone.y
                loss_criterion = nn.MSELoss(reduction = 'none')

                loss_per_var = loss_criterion(out, targets).mean(dim = 0)
                loss = loss_per_var.mean()
                loss_surf_var = loss_criterion(out[data_clone.surf, :], targets[data_clone.surf, :]).mean(dim = 0)
                loss_vol_var = loss_criterion(out[~data_clone.surf, :], targets[~data_clone.surf, :]).mean(dim = 0)
                loss_surf = loss_surf_var.mean()
                loss_vol = loss_vol_var.mean()  

                avg_loss_per_var += loss_per_var.cpu().numpy()
                avg_loss += loss.cpu().numpy()
                avg_loss_surf_var += loss_surf_var.cpu().numpy()
                avg_loss_vol_var += loss_vol_var.cpu().numpy()
                avg_loss_surf += loss_surf.cpu().numpy()
                avg_loss_vol += loss_vol.cpu().numpy()  
                iterNum += 1

                out = out.cpu().data.numpy()
                prediction = self._post_process(out)
                predictions.append(prediction)
        print("Results for test")
        print(avg_loss/iterNum, avg_loss_per_var/iterNum, avg_loss_surf_var/iterNum, avg_loss_vol_var/iterNum, avg_loss_surf/iterNum, avg_loss_vol/iterNum)
        predictions= np.vstack(predictions)
        predictions = dataset.reconstruct_output(predictions)
        return predictions

    def save_model(self, path_model:str,path_scaler:str):
        modelWeight=self.model.state_dict()
        torch.save(modelWeight,path_model)
        self.scaler.save(path_scaler)

    def load_model(self, path_model:str,path_scaler:str):
        model_loader=torch.load(path_model)
        self.model.load_state_dict(model_loader)
        self.model = self.model.to(self.device)
        self.scaler.load(path_scaler)

    def process_dataset(self, dataset, training: bool) -> DataLoader:
        coord_x=dataset.data['x-position']
        coord_y=dataset.data['y-position']
        surf_bool=dataset.extra_data['surface']
        position = np.stack([coord_x,coord_y],axis=1)

        nodes_features,node_labels=dataset.extract_data()
        if training:
            print("Normalize train data")
            nodes_features, node_labels = self.scaler.fit_transform(nodes_features, node_labels)
        else:
            print("Normalize not train data")
            nodes_features, node_labels = self.scaler.transform(nodes_features, node_labels)

        torchDataset=[]
        nb_nodes_in_simulations = dataset.get_simulations_sizes()
        start_index = 0
        for nb_nodes_in_simulation in nb_nodes_in_simulations:
            end_index = start_index+nb_nodes_in_simulation
            simulation_positions = torch.tensor(position[start_index:end_index,:], dtype = torch.float) 
            simulation_features = torch.tensor(nodes_features[start_index:end_index,:], dtype = torch.float) 
            simulation_labels = torch.tensor(node_labels[start_index:end_index,:], dtype = torch.float) 
            simulation_surface = torch.tensor(surf_bool[start_index:end_index])

            sampleData=Data(pos=simulation_positions,
                            x=simulation_features, 
                            y=simulation_labels,
                            surf = simulation_surface.bool()) 
            torchDataset.append(sampleData)
            start_index += nb_nodes_in_simulation
        return DataLoader(dataset=torchDataset,batch_size=1)

    
    def _post_process(self, data):
        try:
            processed = self.scaler.inverse_transform(data)
        except TypeError:
            processed = self.scaler.inverse_transform(data.cpu())
        return processed

Where the global train function is given by

In [ ]:
def global_train(device, train_dataset, network, hparams, criterion = 'MSE', reg = 1):
    model = network.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr = hparams['lr'])
    lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr = hparams['lr'],
            total_steps = (len(train_dataset) // hparams['batch_size'] + 1) * hparams['nb_epochs'],
        )
    start = time.time()

    train_loss_surf_list = []
    train_loss_vol_list = []
    loss_surf_var_list = []
    loss_vol_var_list = []

    pbar_train = tqdm(range(hparams['nb_epochs']), position=0)
    for epoch in pbar_train:        
        train_dataset_sampled = []
        for data in train_dataset:
            data_sampled = data.clone()
            idx = random.sample(range(data_sampled.x.size(0)), hparams['subsampling'])
            idx = torch.tensor(idx)

            data_sampled.pos = data_sampled.pos[idx]
            data_sampled.x = data_sampled.x[idx]
            data_sampled.y = data_sampled.y[idx]
            data_sampled.surf = data_sampled.surf[idx]
            train_dataset_sampled.append(data_sampled)
        train_loader = DataLoader(train_dataset_sampled, batch_size = hparams['batch_size'], shuffle = True)
        del(train_dataset_sampled)

        train_loss, _, loss_surf_var, loss_vol_var, loss_surf, loss_vol = train_model(device, model, train_loader, optimizer, lr_scheduler, criterion, reg = reg)        
        if criterion == 'MSE_weighted':
            train_loss = reg*loss_surf + loss_vol
        del(train_loader)

        train_loss_surf_list.append(loss_surf)
        train_loss_vol_list.append(loss_vol)
        loss_surf_var_list.append(loss_surf_var)
        loss_vol_var_list.append(loss_vol_var)

    loss_surf_var_list = np.array(loss_surf_var_list)
    loss_vol_var_list = np.array(loss_vol_var_list)

    return model

def train_model(device, model, train_loader, optimizer, scheduler, criterion = 'MSE', reg = 1):
    model.train()
    avg_loss_per_var = torch.zeros(4, device = device)
    avg_loss = 0
    avg_loss_surf_var = torch.zeros(4, device = device)
    avg_loss_vol_var = torch.zeros(4, device = device)
    avg_loss_surf = 0
    avg_loss_vol = 0
    iterNum = 0
    
    for data in train_loader:
        data_clone = data.clone()
        data_clone = data_clone.to(device)   
        optimizer.zero_grad()  
        out = model(data_clone)
        targets = data_clone.y

        if criterion == 'MSE' or criterion == 'MSE_weighted':
            loss_criterion = nn.MSELoss(reduction = 'none')
        elif criterion == 'MAE':
            loss_criterion = nn.L1Loss(reduction = 'none')
        loss_per_var = loss_criterion(out, targets).mean(dim = 0)
        total_loss = loss_per_var.mean()
        loss_surf_var = loss_criterion(out[data_clone.surf, :], targets[data_clone.surf, :]).mean(dim = 0)
        loss_vol_var = loss_criterion(out[~data_clone.surf, :], targets[~data_clone.surf, :]).mean(dim = 0)
        loss_surf = loss_surf_var.mean()
        loss_vol = loss_vol_var.mean()

        if criterion == 'MSE_weighted':            
            (loss_vol + reg*loss_surf).backward()           
        else:
            total_loss.backward()
        
        optimizer.step()
        scheduler.step()
        avg_loss_per_var += loss_per_var
        avg_loss += total_loss
        avg_loss_surf_var += loss_surf_var
        avg_loss_vol_var += loss_vol_var
        avg_loss_surf += loss_surf
        avg_loss_vol += loss_vol 
        iterNum += 1

    return avg_loss.cpu().data.numpy()/iterNum, avg_loss_per_var.cpu().data.numpy()/iterNum, avg_loss_surf_var.cpu().data.numpy()/iterNum, avg_loss_vol_var.cpu().data.numpy()/iterNum, \
            avg_loss_surf.cpu().data.numpy()/iterNum, avg_loss_vol.cpu().data.numpy()/iterNum

## Initial step: download the data

In [ ]:
if not os.path.isdir(directory_name):
    download_data(root_path=".", directory_name=directory_name)

#  Benchmark <a id="Case1"></a>

## Loading the datasets

A common dataset will be used for evaluate the augmented simulator. This initial step aims at loading it once and for all.

In [ ]:
benchmark=AirfRANSBenchmark(benchmark_path = DIRECTORY_NAME,
                            config_path = BENCH_CONFIG_PATH,
                            benchmark_name = BENCHMARK_NAME,
                            log_path = LOG_PATH)
benchmark.load(path=DIRECTORY_NAME)

## A baseline "augmented simulator" <a id="bench1-fc"></a>

Along with some dataset, we provide also a baseline (from a trained neural network) with several hyperparameters that can be calibrated accordingly.

In [ ]:
parameters = {"encoder": [7, 64, 64, 8],
               "decoder": [8, 64, 64, 4],
               "nb_hidden_layers": 3,
               "size_hidden_layers": 64,
               "batch_size": 1,
               "nb_epochs": 600,
               "lr": 0.001,
               "bn_bool": True,
               "subsampling": 32000}

mySimulator = AugmentedSimulator(benchmark=benchmark,**parameters)

Training the baseline

In [ ]:
mySimulator.train(benchmark.train_dataset)

It is also possible to save the model weights/scaler normalization.

In [ ]:
mySimulator.save_model("SaveFCModel.pt","SaveScaler")

We are now in position to reload the model/scaler without training it once again

In [ ]:
mySimulator2 = AugmentedSimulator(benchmark=benchmark,**parameters)
mySimulator2.load_model("SaveFCModel.pt","SaveScaler")
prediction_from_reloaded_model = mySimulator2.predict(benchmark._test_dataset)

In [ ]:
prediction_original = mySimulator.predict(benchmark._test_dataset)

for variable in prediction_original.keys():
    np.testing.assert_almost_equal(prediction_original[variable],prediction_from_reloaded_model[variable],1)

Evaluating the baseline on the test and out-of-distribution dataset

In [ ]:
start_test = time.time()
fc_metrics_test = benchmark.evaluate_simulator(dataset="test",augmented_simulator=mySimulator)
test_evaluation_time = time.time() - start_test
test_mean_simulation_time = test_evaluation_time/len(benchmark._test_dataset.get_simulations_sizes())
print("Test evaluation time:",test_evaluation_time)

print("Fully Connected Augmented Simulator")
ml_metrics = fc_metrics_test["test"]["ML"]
print("{:<10} : {}".format("Test ML metrics", ml_metrics))
physical_metrics = fc_metrics_test["test"]["Physics"]
print("{:<10} : {}".format("Test Physical metrics", physical_metrics))

start_test_ood = time.time()
fc_metrics_test_ood = benchmark.evaluate_simulator(dataset="test_ood",augmented_simulator=mySimulator)
test_ood_evaluation_time = time.time() - start_test_ood
test_ood_mean_simulation_time = test_ood_evaluation_time/len(benchmark._test_ood_dataset.get_simulations_sizes())
print("Test ood evaluation time:",test_ood_evaluation_time)

ood_ml_metrics = fc_metrics_test_ood["test_ood"]["ML"]
print("{:<10} : {}".format("OOD ML metrics", ood_ml_metrics))
ood_physical_metrics = fc_metrics_test_ood["test_ood"]["Physics"]
print("{:<10} : {}".format("OOD physical metrics", ood_physical_metrics))

## Compute the score

In order to focus here on the process rather on the precise details, we skip the explanations regarding the score computation. For more details, we refer to [this notebook](https://github.com/IRT-SystemX/ml4physim_startingkit/blob/main/5_Scoring.ipynb)

In [ ]:
def compute_global_score(test_result,ood_result,test_mean_simulation_time,test_ood_mean_simulation_time):
    thresholds={"x-velocity":(0.1,0.2,"min"),
            "y-velocity":(0.1,0.2,"min"),
            "pressure":(0.02,0.1,"min"),
            "pressure_surfacic":(0.08,0.2,"min"),
            "turbulent_viscosity":(0.5,1.0,"min"),
            "mean_relative_drag":(1.0,10.0,"min"),
            "mean_relative_lift":(0.2,0.5,"min"),
            "spearman_correlation_drag":(0.5,0.8,"max"),
            "spearman_correlation_lift":(0.94,0.98,"max")           
    }
    configuration={
        "coefficients":{"ML":0.4,"OOD":0.3,"Physics":0.3},
        "ratioRelevance":{"Speed-up":0.25,"Accuracy":0.75},
        "valueByColor":{"g":2,"o":1,"r":0},
        "maxSpeedRatioAllowed":10000,
        "reference_mean_simulation_time":1500
    }

    ml_metrics = test_result["test"]["ML"]["MSE_normalized"]
    ml_metrics["pressure_surfacic"] = test_result["test"]["ML"]["MSE_normalized_surfacic"]["pressure"]

    phy_variables_to_keep = ["mean_relative_drag","mean_relative_lift","spearman_correlation_drag","spearman_correlation_lift"]
    phy_metrics = {phy_variable:test_result["test"]["Physics"][phy_variable] for phy_variable in phy_variables_to_keep}

    ml_ood_metrics = ood_result["test_ood"]["ML"]["MSE_normalized"]
    ml_ood_metrics["pressure_surfacic"] = ood_result["test_ood"]["ML"]["MSE_normalized_surfacic"]["pressure"]
    phy_ood_metrics = {phy_variable:ood_result["test_ood"]["Physics"][phy_variable] for phy_variable in phy_variables_to_keep}
    ood_metrics = {**ml_ood_metrics,**phy_ood_metrics}

    all_metrics={
        "ML":ml_metrics,
        "Physics":phy_metrics,
        "OOD":ood_metrics
    }

    print(all_metrics)

    reference_mean_simulation_time=configuration["reference_mean_simulation_time"]
    speedUp={
        "ML":reference_mean_simulation_time/test_mean_simulation_time,
         "OOD":reference_mean_simulation_time/test_ood_mean_simulation_time
        }

    accuracyResults=dict()
    for subcategoryName, subcategoryVal in all_metrics.items():
        accuracyResults[subcategoryName]=[]
        for variableName, variableError in subcategoryVal.items():
            thresholdMin,thresholdMax,evalType=thresholds[variableName]
            if evalType=="min":
                if variableError<thresholdMin:
                    accuracyEval="g"
                elif thresholdMin<variableError<thresholdMax:
                    accuracyEval="o"
                else:
                    accuracyEval="r"
            elif evalType=="max":
                if variableError<thresholdMin:
                    accuracyEval="r"
                elif thresholdMin<variableError<thresholdMax:
                    accuracyEval="o"
                else:
                    accuracyEval="g"
    
            accuracyResults[subcategoryName].append(accuracyEval)

    print("accuracyResults: ",accuracyResults)
    
    coefficients=configuration["coefficients"]
    ratioRelevance=configuration["ratioRelevance"]
    valueByColor=configuration["valueByColor"]
    maxSpeedRatioAllowed=configuration["maxSpeedRatioAllowed"]
    mlSubscore=0
    accuracyMaxPoints=ratioRelevance["Accuracy"]
    accuracyResult=sum([valueByColor[color] for color in accuracyResults["ML"]])
    accuracyResult=accuracyResult*accuracyMaxPoints/(len(accuracyResults["ML"])*max(valueByColor.values()))
    mlSubscore+=accuracyResult
    print("ML accuracyResult",accuracyResult)

    speedUpMaxPoints=ratioRelevance["Speed-up"]
    speedUpResult=max(0,min(math.log10(speedUp["ML"])/math.log10(maxSpeedRatioAllowed),1))
    speedUpResult=speedUpResult*speedUpMaxPoints
    mlSubscore+=speedUpResult
    print("ML speedUpResult",speedUpResult)

    accuracyResult=sum([valueByColor[color] for color in accuracyResults["Physics"]])
    accuracyResult=accuracyResult/(len(accuracyResults["Physics"])*max(valueByColor.values()))
    physicsSubscore=accuracyResult
    print("Phy accuracyResult",accuracyResult)

    oodSubscore=0
    accuracyMaxPoints=ratioRelevance["Accuracy"]
    accuracyResult=sum([valueByColor[color] for color in accuracyResults["OOD"]])
    accuracyResult=accuracyResult*accuracyMaxPoints/(len(accuracyResults["OOD"])*max(valueByColor.values()))
    oodSubscore+=accuracyResult
    print("OOD accuracyResult",accuracyResult)

    speedUpMaxPoints=ratioRelevance["Speed-up"]
    speedUpResult=max(0,min(math.log10(speedUp["OOD"])/math.log10(maxSpeedRatioAllowed),1))
    speedUpResult=speedUpResult*speedUpMaxPoints
    oodSubscore+=speedUpResult
    print("OOD speedUpResult",speedUpResult)

    globalScore=100*(coefficients["ML"]*mlSubscore+coefficients["Physics"]*physicsSubscore+coefficients["OOD"]*oodSubscore)
    return globalScore

Finally, we compute the global score

In [ ]:
score = compute_global_score(fc_metrics_test,fc_metrics_test_ood,test_mean_simulation_time,test_ood_mean_simulation_time)
print(score)